# Actividad Guiada 2
#Jorge Luis Olarte Vargas

Link repositorio Github
https://github.com/jolarte8811/Algoritmos_de_Optimizaci-n.git

In [2]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [3]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios
import copy

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos


gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [4]:
problem = tsplib95.load(file)
Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())


In [5]:
def crear_solucion(N):
  solucion = [N[0]]
  for n in N[1:]:
    solucion = solucion + [random.choice(list(set(N)- set({Nodos[0]})-set(solucion)))]
  return(solucion)

def distancia(a,b,problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)



## Busqueda Aleatoria

In [6]:
def Busqueda_Aletoria(problem,N):
  Nodos = list(problem.get_nodes())
  mejor_sol = []
  #mejor_d = 10e10
  mejor_d = float('inf')


  for i in range(N):
    s = crear_solucion(Nodos)

    distancia = distancia_total(s,problem)

    if distancia <mejor_d:
      mejor_sol = s
      mejor_d = distancia

  #print("Mejor solución:" , mejor_sol)
  print("Distancia     :" , mejor_d)
  return mejor_sol

solucion = Busqueda_Aletoria(problem,1000)


Distancia     : 3939


## Busqueda Local

In [7]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]


      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3939
Distancia Mejor Solucion Local: 3718


In [28]:
def buscar(solucion_referencia,mejor_distancia):
  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("  En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("  Distancia     :" , mejor_distancia)
      return (mejor_solucion,mejor_distancia)

    solucion_referencia = vecina



#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem,arranque=100):
  mejor_solucion = []

  #Generar N soluciones iniciales de referencia(aleatoria) y escoje la mejor.
  solucion_referencia = []
  mejor_distancia = 10e10
  solucion_referencia_final = []
  mejor_distancia_final = 10e10

  N_arranque=0
  N_arranque_final=0
  for i in range(arranque):
    solucion_referencia2 = crear_solucion(Nodos)
    mejor_distancia2 = distancia_total(solucion_referencia2, problem)
    if mejor_distancia2 <=mejor_distancia*1.01:
      N_arranque=N_arranque + 1
      print(f"Arranque numero {N_arranque} con distancia inical {mejor_distancia2}")
      solucion_referencia = solucion_referencia2
      mejor_distancia = mejor_distancia2
      mejor_solucion3,mejor_distancia3 = buscar(solucion_referencia,mejor_distancia)

      if mejor_distancia3<mejor_distancia_final:
        solucion_referencia_final = mejor_solucion3
        mejor_distancia_final = mejor_distancia3
        N_arranque_final = N_arranque
  print(f"-------la Mejor solucion fin fue lograda en el arranque {N_arranque_final}--------")
  print(solucion_referencia_final)
  print(f"con distacia {mejor_distancia_final}")


sol = busqueda_local(problem )

Arranque numero 1 con distancia inical 4729
  En la iteracion  35 , la mejor solución encontrada es: [0, 18, 26, 5, 6, 4, 2, 28, 39, 21, 24, 40, 23, 41, 25, 10, 8, 9, 29, 30, 22, 38, 33, 20, 34, 32, 12, 11, 13, 19, 14, 16, 15, 36, 35, 31, 17, 37, 7, 1, 3, 27]
  Distancia     : 1648
Arranque numero 2 con distancia inical 4682
  En la iteracion  38 , la mejor solución encontrada es: [0, 32, 30, 29, 9, 8, 28, 4, 26, 18, 12, 11, 25, 40, 24, 21, 34, 33, 20, 35, 36, 31, 17, 7, 6, 5, 13, 19, 14, 16, 15, 37, 38, 22, 39, 23, 41, 10, 2, 27, 3, 1]
  Distancia     : 1784
Arranque numero 3 con distancia inical 4542
  En la iteracion  29 , la mejor solución encontrada es: [0, 22, 39, 21, 9, 8, 10, 25, 11, 12, 18, 4, 3, 27, 2, 28, 29, 30, 32, 35, 36, 31, 17, 37, 19, 13, 26, 5, 14, 16, 15, 7, 6, 1, 20, 33, 34, 38, 24, 40, 23, 41]
  Distancia     : 1776
Arranque numero 4 con distancia inical 4516
  En la iteracion  31 , la mejor solución encontrada es: [0, 1, 6, 4, 3, 2, 29, 30, 28, 27, 7, 37, 17, 32, 

## Ajustes.
este algoritmo genera 100 soluciones aleatorias, y prueba los vecinos de las "mejores" soluciones iniciales encontradas, considerando una mejor solución la que tenga una distancia menor o igual a 101% del ultimo récord.